In [25]:
import pickle
project = 'qt'
with open('../resource/'+project+'_train.pkl', 'rb') as f_train, open('../resource/'+project+'_test.pkl', 'rb') as f_test:
        tr_data = pickle.load(f_train)
        te_data = pickle.load(f_test)
pred_list = []
kinds = ['train', 'test']
models = ['lgb', 'code_cnn', 'msg_tf']
for kind in kinds:
    for model in models:
        with open('../pred/'+project+'-'+model+'-'+'random'+'-'+kind+'.pkl', 'rb') as f:
            pred_list.append(pickle.load(f))
tr_pred = [[pred_list[0][j], pred_list[1][j], pred_list[2][j]] for j in range(len(pred_list[0]))]
te_pred = [[pred_list[3][j], pred_list[4][j], pred_list[5][j]] for j in range(len(pred_list[3]))]                
tr_data = [tr_pred, tr_data[5]]
te_data = [te_pred, te_data[5]]

In [26]:
pred_list[0]

array([0.07693368, 0.35020531, 0.03905198, ..., 0.18881377, 0.05588778,
       0.09425423])

In [27]:
import numpy as np
from scipy.optimize import minimize
from sklearn.metrics import roc_auc_score

# 仮想的なモデルの出力と真の値（目標値）の設定
probs_model1 = pred_list[0]
probs_model2= pred_list[1]
probs_model3 = pred_list[2]
true_labels = tr_data[1]

# 目的関数の定義（auc）
def objective(weights):
    print(weights)
    weighted_probs = weights[0] * probs_model1 + weights[1] * probs_model2 + weights[2] * probs_model3
    auc = roc_auc_score(true_labels, weighted_probs)
    print(auc)
    return -auc 

# 初期の重みの設定
initial_weights = np.ones(3) / 3  # 3つのモデルのために均等な重みで初期化

# 制約条件
constraints = ({'type': 'ineq', 'fun': lambda x: x},{'type': 'ineq', 'fun': lambda w: np.sum(w) - 1})  # 重みの合計は1になるように制約

# 最適化の実行
result = minimize(objective, initial_weights, constraints=constraints, method='COBYLA')

# 結果の表示
optimal_weights = result.x
print("Optimal Weights:", optimal_weights)


[0.33333333 0.33333333 0.33333333]
0.8090161034770271
[1.33333333 0.33333333 0.33333333]
0.794492585918906
[0.33333333 1.33333333 0.33333333]
0.8080925592604589
[0.33333333 0.33333333 1.33333333]
0.7756239484111135
[0. 1. 0.]
0.7913915404406081
[0.29587585 0.70412415 0.        ]
0.8049319597045813
[0.52678185 0.18018875 0.2930294 ]
0.7992494797764154
[0.26116455 0.26116455 0.26116455]
0.8090161307017483
[0.16279074 0.37830678 0.45890248]
0.8024551090211237
[0.33239371 0.42218775 0.24541855]
0.8115987766517089
[0.4115904  0.43834254 0.15006706]
0.8105970430359134
[0.25106809 0.51625569 0.23267621]
0.811139087234595
[0.36251343 0.44532211 0.29505577]
0.8113360308676066
[0.31895379 0.38627379 0.29477243]
0.8108019362875096
[0.31208931 0.44572202 0.24218867]
0.8116553768470408
[0.32210059 0.45305648 0.23269549]
0.8116951521646839
[0.32411905 0.46961248 0.20626847]
0.8116426356775288
[0.33562789 0.47368563 0.25187915]
0.8116814853546519
[0.33497134 0.44424339 0.23179336]
0.8116945804455393


In [29]:
weighted_average2 = optimal_weights[0] * pred_list[3] + optimal_weights[1] * pred_list[4] + optimal_weights[2] * pred_list[5]
roc_auc_score(te_data[1] ,weighted_average2)

0.7703617401435483

In [1]:
import numpy as np
from scipy.optimize import minimize

# 仮想的なモデルの出力と真の値（目標値）の設定
model1_output = np.array([1.0, 2.0, 3.0])
model2_output = np.array([1.5, 2.5, 3.5])
model3_output = np.array([0.5, 1.5, 2.5])
true_values = np.array([2.0, 3.0, 4.0])

# 目的関数の定義（平均二乗誤差）
def objective(weights):
    weighted_average = weights[0] * model1_output + weights[1] * model2_output + weights[2] * model3_output
    mse = np.mean((weighted_average - true_values)**2)
    return mse

# 初期の重みの設定
initial_weights = np.array([0.33, 0.33, 0.33])  # 初期の重みの総和は1.0になるように設定

# 最適化の実行
result = minimize(objective, initial_weights, method='SLSQP', bounds=[(0, 1)] * 3, constraints={'type': 'eq', 'fun': lambda w: 1.0 - np.sum(w)})

# 結果の表示
optimal_weights = result.x
print("Optimal Weights:", optimal_weights)


Optimal Weights: [0.00000000e+00 1.00000000e+00 1.66533454e-16]


In [11]:
import numpy as np
from scipy.optimize import minimize
from sklearn.metrics import roc_auc_score

# 3つのモデルの確率値
probs_model1 = np.array([0.2, 0.4, 0.6, 0.8])
probs_model2 = np.array([0.3, 0.5, 0.7, 0.9])
probs_model3 = np.array([0.1, 0.3, 0.5, 0.7])

# 真のラベル
true_labels = np.array([0, 1, 0, 1])

# 目的関数の定義
def objective(weights, probs_list, true_labels):
    weighted_probs = np.dot(weights, probs_list)
    auc = roc_auc_score(true_labels, weighted_probs)
    return -auc  # 最大化するためにマイナスをかける

# 初期の重みの設定
initial_weights = np.ones(3) / 3  # 3つのモデルのために均等な重みで初期化

# 制約条件
constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})  # 重みの合計は1になるように制約

# 最適化の実行
result = minimize(objective, initial_weights, args=(probs_model1, probs_model2, probs_model3, true_labels),
                  constraints=constraints)

# 最適な重み
optimal_weights = result.x
print("Optimal Weights:", optimal_weights)

# 最適な重みで計算された確率値
optimal_weighted_probs = np.dot(optimal_weights, [probs_model1, probs_model2, probs_model3])

# 最適な重みで計算されたAUC
optimal_auc = roc_auc_score(true_labels, optimal_weighted_probs)
print("Optimal AUC:", optimal_auc)


TypeError: objective() takes 3 positional arguments but 5 were given